# Can Dilia be a youth?

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import libraries and some constants

import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm
import pandas as pd
import uproot as ur
import atlas_mpl_style as ampl
ampl.use_atlas_style()

In [3]:
# my worspace and directories

path_prefix = '/home/dportill/LCStudies/'
plotpath = path_prefix+'classifier/Plots/'
modelpath = path_prefix+'classifier/Models/'

In [4]:
# import our resolution utilities

import sys
sys.path.append(path_prefix)
from  util import resolution_util as ru
from  util import plot_util as pu
from  util import ml_util as mu

In [5]:
# Load data v7 with images from pubnote

inputpath = '/fast_scratch/atlas_images/v7/'
rootfiles = ["pi0", "piplus", "piminus"]

#setupPionData(inputpath, rootfiles, branches = []) on ml_util with defaultBranches from ClusterTree
trees, pdata = mu.setupPionData(inputpath, rootfiles)

In [6]:
# pdata -> Dictionary of dataframes. "pi0", "piplus", "piminus" are separated inside of the dictionary

np0 = len(pdata['pi0'])
npp = len(pdata['piplus'])
npm = len(pdata['piminus'])

print("Number of pi0 events: {}".format(np0))
print("Number of pi+ events: {}".format(npp))
print("Number of pi- events: {}".format(npm))
print("Total: {}".format(np0+npp+npm))

Number of pi0 events: 263891
Number of pi+ events: 435967
Number of pi- events: 434627
Total: 1134485


In [7]:
pcells = {
    ifile : {
        layer : mu.setupCells(itree, layer, flatten = False)
        for layer in mu.cell_meta
    }
    for ifile, itree in trees.items()
}

In [8]:
# * PRINTING *
# pcells is a dictionary of dictionaries of arrays with keys 'pi0','piplus'and 'piminus' 
#that has the layers as keys: EMB1, EMB2, EMB3, TileBar0, TileBar1, TileBar2 defined on cell_meta
#with dimension of the arrays as len_phi x len_eta

#print(pcells['pi0']['TileBar0']) #neutral pions some no zero values on first TileBar

In [9]:
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow import keras as keras

In [10]:
import tensorflow as tf

In [11]:
##For taking multiple gpus:
#gpu_list = ["/gpu:0","/gpu:1"]
#strategy = tf.distribute.MirroredStrategy(devices=gpu_list)
#ngpu = strategy.num_replicas_in_sync
#print ('Number of devices: {}'.format(ngpu))

In [12]:
# let's just take one gpu:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [13]:
# create train/validation/test subsets containing 70%/10%/20%
# of events from each type of pion event
# merge pi0 and pi+ events
training_classes = ['pi0','piplus']
pdata_merged, pcells_merged, plabels = mu.createTrainingDatasets(training_classes, pdata, pcells)


#pcells_merged -> huge nympy array with all pi0 first and then all pi+ data
#A numpy arraw like this will remove all entries that are not true: pdata_merged.train
#plabels -> which ones are pions plus or pi zero. 

In [14]:
# * PRINTING *

for p_index, plabel in enumerate(training_classes):
    print(" ********************************** p_index: ", p_index, ", plabel: ", plabel)

#pdata after createTrainingDatasets get a 'label' 0 if pi0 
#print(pdata['pi0']['label']==0)
print(pdata['piplus']['label']==1)


 ********************************** p_index:  0 , plabel:  pi0
 ********************************** p_index:  1 , plabel:  piplus
0         True
1         True
2         True
3         True
4         True
          ... 
435962    True
435963    True
435964    True
435965    True
435966    True
Name: label, Length: 435967, dtype: bool


In [15]:
pcellsE23_EMB2G_channels = mu.setupChannelImages(mu.rescaleImages(pcells_merged, (16, 16), layers=['EMB2', 'EMB3']),last=True)

In [16]:
pcellsE1_EMB1G_channels = mu.setupChannelImages(mu.rescaleImages(pcells_merged, (128, 4), layers=['EMB1']),last=True)

In [17]:
pcellsT123_T1G_channels = mu.setupChannelImages(mu.rescaleImages(pcells_merged, (4, 4), layers=['TileBar0', 'TileBar1', 'TileBar2']),last=True)

In [18]:
def merged_model(droprate=.5):
    # EMB1 image (convolutional)
    input1 = Input(shape=(128,4,1), name='emb1_input')
    x1 = Convolution2D(32, (3, 3), padding='same', name='emb1_conv2d_1')(input1)
    x1 = Activation('relu')(x1)
    # x1 = Dropout(droprate)(x1)
    x1 = Convolution2D(32, (3, 3), padding='same', name='emb1_conv2d_2')(x1)
    x1 = Activation('relu')(x1)
    x1 = MaxPool2D(pool_size=(2, 1), padding='same', name='emb1_maxpool_3')(x1)
    x1 = Convolution2D(64, (3, 3), padding='same', name='emb1_conv2d_3')(x1)
    x1 = Activation('relu')(x1)
    #x1 = Dropout(droprate)(x1)
    x1 = Convolution2D(64, (3, 3), padding='same', name='emb1_conv2d_4')(x1)
    x1 = Activation('relu')(x1)
    x1 = MaxPool2D(pool_size=(2, 1), padding='same', name='emb1_maxpool_5')(x1)
    x1 = Convolution2D(128, (2, 2), padding='same', name='emb1_conv2d_6')(x1)
    x1 = Activation('relu')(x1)
    x1 = Convolution2D(128, (2, 2), padding='same', name='emb1_conv2d_7')(x1)
    x1 = Activation('relu')(x1)
    x1 = MaxPool2D(pool_size=(2, 1), padding='same', name='emb1_maxpool_8')(x1)
    x1 = Dropout(droprate, name='emb1_dropout_4')(x1)
    x1 = Flatten(name='emb1_flatten_9')(x1)
    x1 = Dense(128, activation='relu', name='emb1_dense_9')(x1)

    # EMB23 image (convolutional)
    input2 = Input(shape=(16,16,2), name='emb23_input')
    x2 = Convolution2D(32, (1, 1), padding='same', name='emb23_conv1d_1')(input2)
    x2 = Activation('relu')(x2)
    # x2 = Dropout(droprate)(x2)
    x2 = Convolution2D(64, (2, 2), padding='same', name='emb23_conv2d_2')(x2)
    # x2 = Dropout(droprate)(x2)
    x2 = MaxPool2D(pool_size=(2, 2), padding='same', name='emb23_maxpool_3')(x2)
    x2 = Convolution2D(128, (2, 2), padding='same', name='emb23_conv2d_4')(x2)
    x2 = Activation('relu')(x2)
    # x2 = Dropout(droprate)(x2)
    x2 = Convolution2D(128, (2, 2), padding='same', name='emb23_conv2d_5')(x2)
    x2 = Activation('relu')(x2)
    x2 = MaxPool2D(pool_size=(2, 2), padding='same', name='emb23_maxpool_6')(x2)
    x2 = Dropout(droprate, name='emb23_dropout_4')(x2)
    x2 = Flatten(name='emb23_flatten_7')(x2)
    x2 = Dense(128, activation='relu', name='emb23_dense_8')(x2)

    # tiles image (convolutional)
    input3 = Input(shape=(4,4,3), name='tiles_input')
    x3 = Convolution2D(32, (1, 1), padding='same', name='tiles_conv1d_1')(input3)
    x3 = Activation('relu')(x3)
    # x3 = Dropout(droprate)(x3)
    x3 = Convolution2D(64, (2, 2), padding='same', name='tiles_conv2d_2')(x3)
    x3 = Activation('relu')(x3)
    # x3 = Dropout(droprate)(x3)
    x3 = Convolution2D(128, (2, 2), padding='same', name='tiles_conv2d_3')(x3)
    x3 = Activation('relu')(x3)
    x3 = MaxPool2D(pool_size=(2, 2), padding='same', name='tiles_maxpool_4')(x3)
    x3 = Dropout(droprate, name='tiles_dropout_4')(x3)
    x3 = Flatten(name='tiles_flatten_5')(x3)
    x3 = Dense(128, activation='relu', name='tiles_dense_6')(x3)

    # concatenate outputs from the two networks above
    x = concatenate([x1, x2, x3], name='concatenate') 
    x = Dropout(droprate, name='concate_dropout_5')(x)
    x = Dense(64, name='concated_dense_1')(x)    
    x = Activation('relu')(x)
    x = Dropout(droprate, name='dense_dropout_6')(x)

    # final output
    output = Dense(2, activation='softmax', name='dense_output')(x)
    # output = 5*tf.math.tanh(x)   # 0 to +5 range

    model = Model(inputs = [input1, input2, input3], outputs = [output])
    
    # compile model
    # model = multi_gpu_model(model, gpus=4)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [19]:
model_p1_d02 = merged_model(0.2)

In [20]:
#history_p1_d02 = model_p1_d02.fit([pcellsE1_EMB1G_channels[pdata_merged.train], 
#                                pcellsE23_EMB2G_channels[pdata_merged.train], 
#                                pcellsT123_T1G_channels[pdata_merged.train]],
#                                plabels[pdata_merged.train],
#                                validation_data=([pcellsE1_EMB1G_channels[pdata_merged.val], 
#                                                    pcellsE23_EMB2G_channels[pdata_merged.val], 
#                                                    pcellsT123_T1G_channels[pdata_merged.val]], 
#                                                    plabels[pdata_merged.val]),
#                                epochs=50,
#                                batch_size = 128,     #*ngpu,
#                                verbose=2
#                                # use_multiprocessing=False
#                                )

In [21]:
##### Testing less epochs than 250
history_p1_d02 = model_p1_d02.fit([pcellsE1_EMB1G_channels[pdata_merged.train], 
                                pcellsE23_EMB2G_channels[pdata_merged.train], 
                                pcellsT123_T1G_channels[pdata_merged.train]],
                                plabels[pdata_merged.train],
                                validation_data=([pcellsE1_EMB1G_channels[pdata_merged.val], 
                                                    pcellsE23_EMB2G_channels[pdata_merged.val], 
                                                    pcellsT123_T1G_channels[pdata_merged.val]], 
                                                    plabels[pdata_merged.val]),
                                epochs=2,
                                batch_size = 128,
                                verbose=2
                                # use_multiprocessing=False
                                )

Epoch 1/2


NotFoundError:  No execution plan worked!
	 [[node model/tiles_conv1d_1/Conv2D (defined at <ipython-input-21-e8326b59a8c0>:2) ]] [Op:__inference_train_function_2188]

Function call stack:
train_function
